In [ ]:
import myfitnesspal
import datetime
import csv

def init_mfp():
    """Initialize and return a MyFitnessPal client."""
    return myfitnesspal.Client()

def get_meal_data(client, start_date, end_date):
    """Scrape meal data from MyFitnessPal and write it to a CSV file."""
    with open('Data/Cleaned/MFP meals scrapped.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['date', 'meal', 'food', 'quant', 'calories', 'carbs', 'fat', 'protein', 'sodium', 'sugar']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        current_date = start_date
        while current_date <= end_date:
            diary = client.get_date(current_date.year, current_date.month, current_date.day)
            for meal in diary.meals:
                for food in meal.entries:
                    row_data = {
                        'date': current_date.strftime('%Y-%m-%d'),
                        'meal': meal.name,
                        'food': food.name,
                        'quant': food.quantity,
                        'calories': food.nutrition_information['calories'],
                        'carbs': food.nutrition_information['carbohydrates'],
                        'fat': food.nutrition_information['fat'],
                        'protein': food.nutrition_information['protein'],
                        'sodium': food.nutrition_information['sodium'],
                        'sugar': food.nutrition_information['sugar']
                    }
                    writer.writerow(row_data)
            print(f'Data per meal finished {current_date.strftime("%Y-%m-%d")}')
            current_date += datetime.timedelta(days=1)

def get_meal_daily(client, start_date, end_date):
    """Scrape daily summary data from MyFitnessPal and write it to a CSV file."""
    with open('Data/Cleaned/MFP per day scrapped.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['date', 'calories_burned', 'carbs', 'fat', 'protein', 'sodium', 'sugar', 'calories_consumed', 'calories_goal', 'calories_net',
                      'calories_consumed_breakfast', 'calories_consumed_lunch', 'calories_consumed_dinner', 'calories_consumed_snacks']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        current_date = start_date
        while current_date <= end_date:
            diary = client.get_date(current_date.year, current_date.month, current_date.day)
            exercises = diary.exercises[0].entries if diary.exercises else []
            calories_burned = sum(entry.get_as_dict()['nutrition_information']['calories burned'] for entry in exercises if 'calories burned' in entry.get_as_dict()['nutrition_information'])
            calories_meal = {name: diary.meals[i].totals['calories'] if len(diary.meals[i].entries) else 0 for i, name in enumerate(['breakfast', 'lunch', 'dinner', 'snacks'])}
            writer.writerow({
                'date': current_date.strftime('%Y-%m-%d'),
                'calories_burned': calories_burned,
                'carbs': diary.totals['carbohydrates'],
                'fat': diary.totals['fat'],
                'protein': diary.totals['protein'],
                'sodium': diary.totals['sodium'],
                'sugar': diary.totals['sugar'],
                'calories_consumed': diary.totals['calories'],
                'calories_goal': diary.goals['calories'],
                'calories_net': diary.totals['calories'] - diary.goals['calories'],
                'calories_consumed_breakfast': calories_meal['breakfast'],
                'calories_consumed_lunch': calories_meal['lunch'],
                'calories_consumed_dinner': calories_meal['dinner'],
                'calories_consumed_snacks': calories_meal['snacks']
            })
            print(f'Data per day finished {current_date.strftime("%Y-%m-%d")}')
            current_date += datetime.timedelta(days=1)

def main():
    client = init_mfp()
    start_date = datetime.datetime.strptime('2024-03-16', '%Y-%m-%d')
    end_date = datetime.datetime.now() - datetime.timedelta(days=1)
    get_meal_data(client, start_date, end_date)
    get_meal_daily(client, start_date, end_date)

if __name__ == "__main__":
    main()
